Imports

In [13]:
import tensorflow as tf
import numpy as np
import scipy
import os
from tensorflow_model_optimization.python.core.keras.compat import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")



TensorFlow version: 2.17.1
NumPy version: 1.26.4
SciPy version: 1.11.4


In [14]:
if tf.config.list_physical_devices('GPU'):
    print("GPU está conectada")
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        print(f"Nome da GPU: {gpu.name}")
else:
    print("Não conectado a uma GPU")

GPU está conectada
Nome da GPU: /physical_device:GPU:0


Funções auxiliares

In [15]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, average_precision_score
from scipy.stats import ks_2samp
#import scikitplot as skplt
import matplotlib.pyplot as plt
from numpy import interp

from scipy.stats import ks_2samp
from sklearn.metrics import roc_curve, auc


def extract_final_losses(history):
    """Função para extrair o melhor loss de treino e validação.

    Argumento(s):
    history -- Objeto retornado pela função fit do keras.

    Retorno:
    Dicionário contendo o melhor loss de treino e de validação baseado
    no menor loss de validação.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    idx_min_val_loss = np.argmin(val_loss)
    return {'train_loss': train_loss[idx_min_val_loss], 'val_loss': val_loss[idx_min_val_loss]}

def plot_training_error_curves(history):
    """Função para plotar as curvas de erro do treinamento da rede neural.

    Argumento(s):
    history -- Objeto retornado pela função fit do keras.

    Retorno:
    A função gera o gráfico do treino da rede e retorna None.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    fig, ax = plt.subplots()
    ax.plot(train_loss, label='Train')
    ax.plot(val_loss, label='Validation')
    ax.set(title='Training and Validation Error Curves', xlabel='Epochs', ylabel='Loss (MSE)')
    ax.legend()
    plt.show()

def compute_performance_metrics(y, y_pred_class, y_pred_scores=None):
    accuracy = accuracy_score(y, y_pred_class)
    recall = recall_score(y, y_pred_class)
    precision = precision_score(y, y_pred_class)
    f1 = f1_score(y, y_pred_class)
    performance_metrics = (accuracy, recall, precision, f1)
    if y_pred_scores is not None:
        # skplt.metrics.plot_ks_statistic(y, y_pred_scores)
        # plt.savefig("ks_plot.png")
        # plt.show()
        y_pred_scores = y_pred_scores[:, 1]
        auroc = roc_auc_score(y, y_pred_scores)
        aupr = average_precision_score(y, y_pred_scores)
        performance_metrics = performance_metrics + (auroc, aupr)
    return performance_metrics

def print_metrics_summary(accuracy, recall, precision, f1, auroc=None, aupr=None):
    print()
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1))
    if auroc is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=auroc))
    if aupr is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUPR:", value=aupr))

Definindo paths e carregabdo os dados

In [16]:
path = "/home/pedro/projetoDL/dataset/processado/"
save_path = "/home/pedro/projetoDL/log/prunning/fold3/"

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(save_path):
    os.makedirs(save_path)

Avaliando no conj de teste 

In [17]:
path = "/home/pedro/projetoDL/dataset/processado/"

Y = np.load(path + 'Y_test_Driving_NewApproach_Injected_v2.npz')
Y= Y.f.arr_0

X = np.load(path + 'X_test_Driving_NewApproach_Injected_v2.npz')
X = X.f.arr_0

from sklearn.model_selection import train_test_split

# Definir 2/3 dos dados
X_subset, _, Y_subset, _ = train_test_split(
    X, Y, train_size=1/3, stratify=Y, random_state=42  # Stratify para manter a proporção de classes
)

# Agora X_subset e Y_subset têm 2/3 dos dados originais
X = X_subset
Y = Y_subset

Modelo Prunned

In [18]:
import time 
import os
import seaborn as sns

i = 1
tmp_path = '/home/pedro/projetoDL/log/prunning/fold3/'

for fold_no in [3]:
  load_path = tmp_path
  save_path = tmp_path + 'results_prunned/'
  print(load_path)

  # Verifica se o diretório existe, se não, cria o diretório
  if not os.path.exists(save_path):
      os.makedirs(save_path)

  file1 = open(save_path + 'TESTE_model_' + str(fold_no) + '_log.txt', "a")

  model = keras.models.load_model(load_path + 'fold3_only_3epoch_prunned.h5')

  start = time.time()
  y_pred_scores = model.predict(X)
  end = time.time()

  print(f"\nSubset size: 2/3\n", file=file1)

  print(f"Runtime of the program is {end - start}", file=file1)

  total_time = end - start
  timesample = total_time/y_pred_scores.shape[0]

  print(f"us/sample is {timesample*1000000}", file=file1)

  y_pred_class = (y_pred_scores > 0.5).astype("int64")

  y_pred_scores2 = y_pred_scores
  y_pred_scores_0 = 1 - y_pred_scores
  y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

  accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(Y, y_pred_class, y_pred_scores)
       
  print(f'Results for fold {fold_no}: Recall of {recall}; accuracy of {accuracy}; precision of {precision}; f1 of {f1}; auroc of {auroc}; aupr of {aupr}', file=file1)
  
  fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y, y_pred_scores2)
  auc_keras = auc(fpr_keras, tpr_keras)

  sns.set(rc={'figure.figsize':(8,8)})
  plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
  plt.plot(fpr_keras, tpr_keras, marker='.', label='MLP (auc = %0.4f)' % auc_keras)
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('ROC')
  plt.legend(loc="lower right")
  plt.savefig(save_path + "plot_roc_TESTE.png")
  plt.show()     

  file1.close()
  del model
  time.sleep(3)

/home/pedro/projetoDL/log/prunning/fold3/


2024-11-24 19:50:12.258662: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3182405248 exceeds 10% of free system memory.
2024-11-24 19:50:23.130295: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.96GiB (rounded to 3182405376)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-11-24 19:50:23.130325: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-11-24 19:50:23.130337: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 52, Chunks in use: 50. 13.0KiB allocated for chunks. 12.5KiB in use in bin. 7.2KiB client-requested in use in bin.
2024-11-24 19:50:23.130344: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

4 19:50:23.130520: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000700 of size 256 next 4
2024-11-24 19:50:23.130525: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000800 of size 256 next 6
2024-11-24 19:50:23.130530: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000900 of size 256 next 7
2024-11-24 19:50:23.130535: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000a00 of size 256 next 5
2024-11-24 19:50:23.130540: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000b00 of size 256 next 8
2024-11-24 19:50:23.130545: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000c00 of size 256 next 11
2024-11-24 19:50:23.130550: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000d00 of size 256 next 12
2024-11-24 19:50:23.130555: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7a9666000e00 of size 25